In [1]:
import os

os.chdir('/Users/edwardchiu/Desktop/projects/baha-anime-analysis')
os.getcwd()

'/Users/edwardchiu/Desktop/projects/baha-anime-analysis'

In [8]:
import gspread
import gspread.utils
from datetime import datetime
import pytz
from google.oauth2.service_account import Credentials
import modules.settings as settings
import modules.settings as settings
from gspread_formatting import (
    get_frozen_row_count,
    get_frozen_column_count,
    set_frozen,
    set_row_height
)
import pandas as pd

In [3]:
# Authenticate with Google
creds = Credentials.from_service_account_file(settings.service_account_file, scopes=settings.scope)
client = gspread.authorize(creds)

In [4]:
# Open the Google Sheet
spreadsheet = client.open(settings.sheetname)
worksheet = spreadsheet.worksheet(settings.tabnames[1])

In [ ]:
worksheet.update_acell('C1', 'yes!!!')

In [ ]:
get_frozen_row_count(worksheet)

In [ ]:
from gspread_formatting import set_row_heights, set_row_height

In [ ]:
set_row_height(worksheet, '5:9', 1000)

In [ ]:
get_frozen_column_count(worksheet)

In [7]:
df_all_episode = pd.read_csv('data/all_episode.csv')
df_all_episode.head()

,anime_name,episode_name,episode_link,uploaded_time,view,comment_count,danmu_count
0,青之壬生浪,[1],https://ani.gamer.com.tw/animeVideo.php?sn=40525,2024-10-19 18:00,18000.0,31,134
1,青之壬生浪,[2],https://ani.gamer.com.tw/animeVideo.php?sn=40526,2024-10-26 18:00,9411.0,10,61
2,青之壬生浪,[3],https://ani.gamer.com.tw/animeVideo.php?sn=40580,2024-11-02 18:00,6432.0,4,26
3,青之壬生浪,[4],https://ani.gamer.com.tw/animeVideo.php?sn=40614,2024-11-09 18:00,5217.0,4,27
4,青之壬生浪,[5],https://ani.gamer.com.tw/animeVideo.php?sn=40683,2024-11-16 18:00,3919.0,3,21


In [9]:
df_all_episode = pd.read_csv('data/all_episode.csv')
print('Finish extracting data from web, start import data to spreadsheet...')
column_names = settings.column_names['episode_level']

df_all_episode = df_all_episode.fillna('')


# Step 1: Convert all values to string and handle encoding issues
def convert_to_utf8(val):
    if isinstance(val, str):
        return val.encode('utf-8', errors='ignore').decode('utf-8')
    return str(val)


# Create hyperlink format for Google Sheets
def create_hyperlink(name, url):
    return f'=HYPERLINK("{url}", "{name}")'


# Apply the hyperlink function to each row in the DataFrame
df_all_episode['episode_name'] = df_all_episode.apply(
    lambda row: create_hyperlink(row['episode_name'], row['episode_link']), axis=1)

df_all_episode = df_all_episode[list(column_names.keys())]
df_all_episode = df_all_episode.map(convert_to_utf8)

# worksheet = self.spreadsheet.worksheet(settings.tabnames[1])

# Get existing frozen rows
# frozen_rows = get_frozen_row_count(worksheet) or 4
frozen_rows = 4

# Clean sheet
worksheet.clear_basic_filter()
worksheet.batch_clear(
    [f'A4:{gspread.utils.rowcol_to_a1(worksheet.row_count, worksheet.col_count)}'])

header = [list(column_names.values())]
values = df_all_episode.values.tolist()

END = len(header + values) + 3

range_end = gspread.utils.rowcol_to_a1(END, len(header[0]))
my_range = f'A4:{range_end}'
worksheet.batch_clear([my_range])
worksheet.update(range_name=my_range, values=header + values, raw=False)
if END > frozen_rows:
    worksheet.resize(rows=END)

# Re-apply existing formats
set_frozen(worksheet=worksheet,
           rows=frozen_rows)

# Add filter
worksheet.set_basic_filter(my_range)

# Updated time and latest data source file
now = datetime.now(pytz.utc).strftime('%Y-%m-%d %H:%M:%S %Z')
worksheet.update_acell('B1', now)

Finish extracting data from web, start import data to spreadsheet...


{'spreadsheetId': '1F94CV-TTa628TumABt3DOF_beqJxQTJ-Mjp1nHkWQDE',
 'updatedRange': "'Episode-Level Data'!B1",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}